In [1]:
import tensorflow as tf
import numpy as np
from collections import Counter
import os
import datetime

In [2]:
def read_signals(filename):
    with open(filename, 'r') as fp:
        data = fp.read().splitlines()
        data = map(lambda x: x.rstrip().lstrip().split(), data)
        data = [list(map(float, line)) for line in data]
        data = np.array(data, dtype=np.float32)
    return data


def read_labels(filename):
    with open(filename, 'r') as fp:
        activities = fp.read().splitlines()
        activities = list(map(int, activities))
    return np.array(activities)


def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation, :, :]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels


def one_hot_encode(np_array, num_labels):
    return (np.arange(num_labels) == np_array[:, None]).astype(np.float32)


def reformat_data(dataset, labels):
    no_labels = len(np.unique(labels))
    labels = one_hot_encode(labels, no_labels)
    dataset, labels = randomize(dataset, labels)
    return dataset, labels


d_activity_num_to_labels = {
    1: 'walking',
    2: 'walking upstairs',
    3: 'walking downstairs',
    4: 'sitting',
    5: 'standing',
    6: 'laying'
}


# Loading the dataset

In [3]:
INPUT_FOLDER_TRAIN = 'UCI HAR Dataset/train/Inertial Signals/'
INPUT_FOLDER_TEST = 'UCI HAR Dataset/test/Inertial Signals/'

INPUT_FILES_TRAIN = [
    'body_acc_x_train.txt', 'body_acc_y_train.txt', 'body_acc_z_train.txt',
    'body_gyro_x_train.txt', 'body_gyro_y_train.txt', 'body_gyro_z_train.txt',
    'total_acc_x_train.txt', 'total_acc_y_train.txt', 'total_acc_z_train.txt'
]

INPUT_FILES_TEST = [
    'body_acc_x_test.txt', 'body_acc_y_test.txt', 'body_acc_z_test.txt',
    'body_gyro_x_test.txt', 'body_gyro_y_test.txt', 'body_gyro_z_test.txt',
    'total_acc_x_test.txt', 'total_acc_y_test.txt', 'total_acc_z_test.txt'
]

LABELFILE_TRAIN = 'UCI HAR Dataset/train/y_train.txt'
LABELFILE_TEST = 'UCI HAR Dataset/test/y_test.txt'

train_signals, test_signals = [], []

for input_file in INPUT_FILES_TRAIN:
    signal = read_signals(INPUT_FOLDER_TRAIN + input_file)
    train_signals.append(signal)
train_signals = np.transpose(np.array(train_signals), (1, 2, 0))

for input_file in INPUT_FILES_TEST:
    signal = read_signals(INPUT_FOLDER_TEST + input_file)
    test_signals.append(signal)
test_signals = np.transpose(np.array(test_signals), (1, 2, 0))

train_labels = read_labels(LABELFILE_TRAIN)
test_labels = read_labels(LABELFILE_TEST)

[no_signals_train, no_steps_train,
 no_components_train] = np.shape(train_signals)
[no_signals_test, no_steps_test, no_components_test] = np.shape(test_signals)
no_labels = len(np.unique(train_labels[:]))

print(
    "The train dataset contains {} signals, each one of length {} and {} components ".
    format(no_signals_train, no_steps_train, no_components_train))
print(
    "The test dataset contains {} signals, each one of length {} and {} components ".
    format(no_signals_test, no_steps_test, no_components_test))
print(
    "The train dataset contains {} labels, with the following distribution:\n {}".
    format(np.shape(train_labels)[0], Counter(train_labels[:])))
print(
    "The test dataset contains {} labels, with the following distribution:\n {}".
    format(np.shape(test_labels)[0], Counter(test_labels[:])))

train_data, train_labels = reformat_data(train_signals, train_labels)
test_data, test_labels = reformat_data(test_signals, test_labels)

The train dataset contains 7352 signals, each one of length 128 and 9 components 
The test dataset contains 2947 signals, each one of length 128 and 9 components 
The train dataset contains 7352 labels, with the following distribution:
 Counter({6: 1407, 5: 1374, 4: 1286, 1: 1226, 2: 1073, 3: 986})
The test dataset contains 2947 labels, with the following distribution:
 Counter({6: 537, 5: 532, 1: 496, 4: 491, 2: 471, 3: 420})


# Building and training the model

Below my solution. Changes:
- model constructed inline, removed the original fccd model ... (just preference)
+ **multi-layer RNN**
+ **dropout**
+ calculating everything in tensorflow (including accuracy)
+ **tensorboard** logging, last point is run on the test set (nice for comparing runs)
+ testing just once after training on the full test set
+ tuned hyperparameters, for test loss above **70%**
- model still overfits (;/)

## Set Hyperparameters

In [ ]:
# dataset 
NUM_STEPS = 128
NUM_COMPONENTS = 9
NUM_LABLES = 6

# network architecture
NUM_UNITS = 256
NUM_LAYERS = 3
DROPOUT = True

# training
BATCH_SIZE = 32
LEARNING_RATE = 0.00005
TOTAL_STEPS = 5000
LOG_STEP = 100
DISPLAY_STEP = 100

## Build model

In [ ]:
graph = tf.Graph()
with graph.as_default():

    # First make placeholders for data.
    # Use None for batch size, to be more flexible.
    # That will allow for testing on full test set
    tf_dataset = tf.placeholder(tf.float32, shape=[None, NUM_STEPS, NUM_COMPONENTS])
    tf_labels = tf.placeholder(tf.float32, shape=[None, NUM_LABLES])

    # Build the model
    data = tf.unstack(tf_dataset, axis=1)

    cell = tf.contrib.rnn.BasicLSTMCell(NUM_UNITS)
    if DROPOUT:
        cell = tf.contrib.rnn.DropoutWrapper(cell, 0.25)
    cell = tf.contrib.rnn.MultiRNNCell(
        [tf.contrib.rnn.BasicLSTMCell(NUM_UNITS) for _ in range(NUM_LAYERS)])

    output, state = tf.contrib.rnn.static_rnn(cell, data, dtype=tf.float32)

    logits = tf.layers.dense(output[-1], 6)

    prediction = tf.nn.softmax(logits)
    correct_prediction = tf.equal(
        tf.argmax(logits, 1), tf.argmax(tf_labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, labels=tf_labels))

    optimizer = tf.train.AdamOptimizer(
        learning_rate=LEARNING_RATE).minimize(loss)

    tf.summary.scalar('loss', loss)
    tf.summary.scalar('accuracy', accuracy)
    summ = tf.summary.merge_all()

## Run training

In [ ]:
# construct the tensorboard run name
now = datetime.datetime.now()
LOGDIR = ("/tmp/hack_11_sec/" +
          '{:02}:{:02}'.format(now.hour, now.minute) +
          '_bs{}'.format(BATCH_SIZE) + 
          '_un{}'.format(NUM_UNITS) + 
          '_la{}'.format(NUM_LAYERS) + 
          '_lr{}'.format(LEARNING_RATE))
if DROPOUT: LOGDIR += '_d'

In [4]:
# train and test the model
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized with learning rate of ', LEARNING_RATE)

    writer = tf.summary.FileWriter(LOGDIR)
    writer.add_graph(session.graph)

    for step in range(TOTAL_STEPS):

        offset = (step * BATCH_SIZE) % (train_labels.shape[0] - BATCH_SIZE)
        batch_data = train_data[offset:(offset + BATCH_SIZE), :, :]
        batch_labels = train_labels[offset:(offset + BATCH_SIZE), :]

        # train the weights
        feed_dict = {tf_dataset: batch_data, tf_labels: batch_labels}
        session.run([optimizer], feed_dict)

        # log for tensorboard
        if step % LOG_STEP == 0:
            _, s = session.run([loss, summ], feed_dict)
            writer.add_summary(s, step)
            writer.flush()

        # print during training
        if step % DISPLAY_STEP == 0:
            l, train_accuracy = session.run([loss, accuracy], feed_dict)
            message = "step {:4} : loss is {:6.2f}, accuracy on training set {:6.2f} %".format(
                step, l, train_accuracy * 100)
            print(message)

    # calculate and print the accuract on the full test set
    feed_dict = {tf_dataset: test_data, tf_labels: test_labels}
    l, test_accuracy, s = session.run([loss, accuracy, summ], feed_dict)
    writer.add_summary(s, step)
    writer.flush()
    message = "Test loss is {:6.2f}, accuracy on training set {:6.2f} %".format(
        l, test_accuracy * 100)
    print(message)

Initialized with learning rate of  5e-05
step    0 : loss is   1.52, accuracy on training set   0.00 %
step  100 : loss is   0.95, accuracy on training set  40.62 %
step  200 : loss is   0.91, accuracy on training set  43.75 %
step  300 : loss is   0.88, accuracy on training set  28.12 %
step  400 : loss is   0.78, accuracy on training set  50.00 %
step  500 : loss is   0.65, accuracy on training set  68.75 %
step  600 : loss is   0.77, accuracy on training set  43.75 %
step  700 : loss is   0.59, accuracy on training set  62.50 %
step  800 : loss is   0.54, accuracy on training set  53.12 %
step  900 : loss is   0.36, accuracy on training set  56.25 %
step 1000 : loss is   0.68, accuracy on training set  56.25 %
step 1100 : loss is   0.56, accuracy on training set  59.38 %
step 1200 : loss is   0.42, accuracy on training set  50.00 %
step 1300 : loss is   0.40, accuracy on training set  62.50 %
step 1400 : loss is   0.33, accuracy on training set  75.00 %
step 1500 : loss is   0.33, a